In [1]:
!pip install ultralytics --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 12.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-10.3.

In [2]:
# Cell 1: Setup and Imports
import os
import shutil
from tqdm import tqdm 
import cv2
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from itertools import groupby
from PIL import Image
from ultralytics import YOLO

# Create directory structure
os.makedirs('/kaggle/working/yolo_dataset/images/train', exist_ok=True)
os.makedirs('/kaggle/working/yolo_dataset/labels/train', exist_ok=True)
os.makedirs('/kaggle/working/yolo_dataset/images/val', exist_ok=True)
os.makedirs('/kaggle/working/yolo_dataset/labels/val', exist_ok=True)
os.makedirs('/kaggle/working/kaggle_labels', exist_ok=True)
os.makedirs('/kaggle/working/kaggle_binarized', exist_ok=True)
os.makedirs('/kaggle/working/trained_models', exist_ok=True)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
def preprocess_image(img_path, output_binarized_dir, output_coords_dir, class_id):
    """Preprocess a Kaggle image using the same logic as your original script."""
    img = cv2.imread(img_path)
    if img is None:
        print(f"Warning: Unable to read image: {img_path}")
        return None, None

    # Get file name base
    filename = os.path.basename(img_path)
    base_name = os.path.splitext(filename)[0]

    # 1. Grayscale and binarize using Otsu's method
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Save binarized image
    binarized_filename = f"bin_{base_name}.png"
    binarized_path = os.path.join(output_binarized_dir, binarized_filename)
    cv2.imwrite(binarized_path, binary)

    # 2. Word segmentation via dilation + contour detection
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 5))  # Same kernel as your script
    dilated = cv2.dilate(binary, kernel, iterations=1)
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    word_boxes = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w > 10 and h > 10:  # Noise filter
            word_boxes.append([x, y, x + w, y + h])

    # Sort boxes: top to bottom, then left to right within line buckets
    word_boxes.sort(key=lambda box: (box[1] // 50, box[0]))

    # 3. Save YOLO .txt labels
    yolo_lines = []
    width, height = img.shape[1], img.shape[0]

    for box in word_boxes:
        x1, y1, x2, y2 = box
        x_center = ((x1 + x2) / 2) / width
        y_center = ((y1 + y2) / 2) / height
        box_width = (x2 - x1) / width
        box_height = (y2 - y1) / height
        yolo_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}")

    label_filename = f"{base_name}.txt"
    label_path = os.path.join(output_coords_dir, label_filename)
    with open(label_path, 'w') as f:
        f.write('\n'.join(yolo_lines))

    return binarized_filename, label_filename


In [4]:

# Cell 3: Dataset Preparation
def prepare_dataset():

    # Configure paths
    YOUR_BINARIZED_PATH = '/kaggle/input/donna-handwriting/preprocessing/binarized'
    YOUR_COORDINATES_PATH = '/kaggle/input/donna-handwriting/preprocessing/labels'
    KAGGLE_TRAIN_PATH = '/kaggle/input/handwriting-recognition/train_v2/train'
    OUTPUT_PATH = '/kaggle/working/yolo_dataset'
    
    # Create processing directories
    KAGGLE_BINARIZED_PATH = '/kaggle/working/kaggle_binarized'
    KAGGLE_LABELS_PATH = '/kaggle/working/kaggle_labels'
    os.makedirs(KAGGLE_BINARIZED_PATH, exist_ok=True)
    os.makedirs(KAGGLE_LABELS_PATH, exist_ok=True)

    # 1. Process YOUR data (class 1)
    your_images = [f for f in os.listdir(YOUR_BINARIZED_PATH) 
                  if f.startswith('bin_') and f.endswith('.png')]
    your_labels = []

    # Copy and rename your existing labels to match image names
    for img in your_images:
        base_name = img.replace('.png', '')  # Already includes 'bin_'
        label_name = f"{base_name}.txt"
        original_base = base_name.replace('bin_', '')  # Get original file name for label
        src = os.path.join(YOUR_COORDINATES_PATH, f"{original_base}.txt")
        dst = os.path.join(KAGGLE_LABELS_PATH, label_name)
        if os.path.exists(src):
            shutil.copy(src, dst)
            your_labels.append(label_name)
        else:
            print(f"Label missing for {img}, skipping.")

    # 2. Process KAGGLE data (class 0)
    all_kaggle_images = [f for f in os.listdir(KAGGLE_TRAIN_PATH) if f.endswith('.jpg')][:10000]
    kaggle_images = []
    kaggle_labels = []
    
    for img_file in tqdm(all_kaggle_images, desc="Processing Kaggle"):
        processed_img, label_file = preprocess_image(
            os.path.join(KAGGLE_TRAIN_PATH, img_file),
            KAGGLE_BINARIZED_PATH,
            KAGGLE_LABELS_PATH,
            class_id=0
        )
        if processed_img:
            kaggle_images.append(processed_img)
            kaggle_labels.append(label_file)

    print(f"Your preprocessed images: {len(your_images)}")
    print(f"Kaggle processed images: {len(kaggle_images)}")

    # Create YOLO directory structure
    for subset in ['train', 'val']:
        os.makedirs(f'{OUTPUT_PATH}/images/{subset}', exist_ok=True)
        os.makedirs(f'{OUTPUT_PATH}/labels/{subset}', exist_ok=True)

    def copy_files(files, labels, src_img_dir, src_lbl_dir, subset):
        copied = 0
        for img, lbl in zip(files, labels):
            try:
                # Copy image
                shutil.copy(
                    os.path.join(src_img_dir, img),
                    os.path.join(f"{OUTPUT_PATH}/images/{subset}", img)
                )
                # Copy label
                shutil.copy(
                    os.path.join(src_lbl_dir, lbl),
                    os.path.join(f"{OUTPUT_PATH}/labels/{subset}", lbl)
                )
                copied += 1
            except Exception as e:
                print(f"Error copying {img} & {lbl}: {str(e)}")
        return copied

    def process_dataset(images, labels, src_img_dir, src_lbl_dir):
        if images:
            train_img, val_img, train_lbl, val_lbl = train_test_split(
                images, labels, test_size=0.2, random_state=42
            )
            return (
                copy_files(train_img, train_lbl, src_img_dir, src_lbl_dir, 'train'),
                copy_files(val_img, val_lbl, src_img_dir, src_lbl_dir, 'val')
            )
        return (0, 0)

    # Process datasets
    prep_train, prep_val = process_dataset(your_images, your_labels, YOUR_BINARIZED_PATH, KAGGLE_LABELS_PATH)
    kaggle_train, kaggle_val = process_dataset(kaggle_images, kaggle_labels, KAGGLE_BINARIZED_PATH, KAGGLE_LABELS_PATH)

    # Results
    print("\n=== Results ===")
    print(f"Your Data - Train: {prep_train}, Val: {prep_val}")
    print(f"Kaggle Data - Train: {kaggle_train}, Val: {kaggle_val}")
    print(f"Total - Train: {prep_train + kaggle_train}, Val: {prep_val + kaggle_val}")

# Run preparation
prepare_dataset()


Processing Kaggle: 100%|██████████| 10000/10000 [02:13<00:00, 74.96it/s]


Your preprocessed images: 20
Kaggle processed images: 10000

=== Results ===
Your Data - Train: 16, Val: 4
Kaggle Data - Train: 8000, Val: 2000
Total - Train: 8016, Val: 2004


In [5]:
# Cell 4: Create data.yaml
yaml_content = """
train: /kaggle/working/yolo_dataset/images/train
val: /kaggle/working/yolo_dataset/images/val

nc: 2
names: ['kaggle_handwriting', 'my_handwriting']
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as f:
    f.write(yaml_content.strip())

print("data.yaml created!")

data.yaml created!


In [6]:
# Cell 5: Validation
def validate_dataset():
    print("\nDataset structure:")
    !ls /kaggle/working/yolo_dataset
    print("\nSample train images:")
    !ls /kaggle/working/yolo_dataset/images/train | head -5
    print("\nSample train labels:")
    !ls /kaggle/working/yolo_dataset/labels/train | head -5
    
    # Verify label contents
    sample_label = os.listdir('/kaggle/working/yolo_dataset/labels/train')[0]
    print(f"\nSample label contents ({sample_label}):")
    !cat /kaggle/working/yolo_dataset/labels/train/{sample_label}

validate_dataset()


Dataset structure:
data.yaml  images  labels

Sample train images:
bin_p10.png
bin_p11.png
bin_p12.png
bin_p13.png
bin_p14.png
ls: write error: Broken pipe

Sample train labels:
bin_p10.txt
bin_p11.txt
bin_p12.txt
bin_p13.txt
bin_p14.txt
ls: write error: Broken pipe

Sample label contents (TRAIN_121790.txt):
0 0.283505 0.500000 0.567010 1.000000

In [7]:
# Cell 6: Training
model = YOLO('yolov8n.pt')  # Start with small model

100%|██████████| 6.25M/6.25M [00:00<00:00, 79.6MB/s]


In [8]:
# Cell 7: Start Training
results = model.train(
    data='/kaggle/working/yolo_dataset/data.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    patience=0,
    device='0',
    name='handwriting_detection'
)

Ultralytics 8.3.141 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=handwriting_detection, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=0, perspective=0.0, plots=True, 

100%|██████████| 755k/755k [00:00<00:00, 16.5MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 71.4MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 25.0±11.4 MB/s, size: 0.6 KB)


...


50 epochs completed in 0.975 hours.
Optimizer stripped from runs/detect/handwriting_detection/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/handwriting_detection/weights/best.pt, 6.2MB

Validating runs/detect/handwriting_detection/weights/best.pt...
Ultralytics 8.3.141 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [00:02<00:00, 30.11it/s]


                   all       2004        797      0.984      0.942      0.966      0.797
        my_handwriting          4        797      0.984      0.942      0.966      0.797


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.0ms preprocess, 0.4ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/handwriting_detection
